# DSPy Take 1

In this exercise, we learn DSPy from the very basic - hello world style then go furture. The goal is to understand what it is exactly

In [1]:
!pip3 install --quiet openai python-dotenv dspy-ai


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [2]:
!pip3 install --quiet arize-phoenix openinference-instrumentation-dspy opentelemetry-exporter-otlp


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


## DSPy provides its own tracing, but here we use Arize Phoenix ##

In [ ]:
import phoenix as px

px.launch_app()

## Set up Phoenix instrumentor ##

In [5]:
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

## Hello World ##

In [6]:
import dspy, os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
lm = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=4000)
dspy.settings.configure(lm=lm)

OpenAI API Key: ········


In [7]:
predictor = dspy.Predict("question -> answer")
print(predictor(question="hello?"))

Prediction(
    answer='Hello! How can I assist you today?'
)


**What is happening?**
![hello](./tracing_hello.jpg)

In [41]:
print(predictor(question="what is the capital city of France?"))

Prediction(
    answer='Paris'
)


![france](./tracing_france.jpg)

**So DSPy can decide the more appropriate prompt based on the question**

## Next, a QA module ##

Looking into the "example" folder in DSPy, I think the "BasicQA" would be a good starting point. And copied over below. 

However, I changed the "desc" from ""often between 1 and 5 words"" to "whatever" because ... I am curious about it

In [9]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="whatever") # used to be often between 1 and 5 words"
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = dspy.Predict(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

In [43]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Question: what is the capital city of France??\nAnswer: Paris'

![whatever](./tracing_whatever.jpg)

So, the desc ("whatever") is being used along with the "answer" field in the prompt

**Now let's give it a proper description, like "the city name"?**

In [44]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="the city name")

In [45]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'the city name'

![the city name desc](./tracing_desc.jpg)

**Ok let's get back to the "good" one**

In [23]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 2 words")

In [24]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Paris'

**But what if we don't specify the desc**

In [48]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField() # no desc

In [49]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Question: what is the capital city of France??\nAnswer: Paris'

In [50]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="the answer") # being naive again

In [52]:
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Question: what is the capital city of France??\nAnswer: Paris'

**enough of desc**

In [32]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 2 words, or N/A")

In [33]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of foiadhfioah;fg;hdasof??")
pred.answer

'N/A'

**to be fair, above experiments only demonstrate the unpredicable behavior how LLM react to our prompt, it is more about LLM than DSPy**

## Now Let's Specify JSON output ##

Based on document:
> You can specify JSON-type descriptions in the `desc` field of the long-form signature `dspy.OutputField` (e.g. `output = dspy.OutputField(desc='key-value pairs')`).
>
> If you notice outputs are still not conforming to JSON formatting, try Asserting this constraint! Check out [Assertions](https://dspy-docs.vercel.app/docs/> building-blocks/assertions) (or the next question!)


In [12]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="JSON type, with key being question and value being 1 or 2 words, or empty if not answer")
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France?")
pred.answer

'{\n  "question": "Paris"\n}'

It works but in practice we'd often need sort of schema. There's been some discussion on this topic at https://github.com/stanfordnlp/dspy/issues/264 and the solution is to use the Pydantic Type https://github.com/stanfordnlp/dspy?tab=readme-ov-file#5-pydantic-types

In [13]:
!pip3 install --quiet pydantic


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


**Here are the changes**
1. create a new Country class to hold the data, like country name ad capital cityname
2. change BasicQA to specify answer is of type Country
3. change  BasicQABot to use "TypedPredictor" from the original "Predict"

In [15]:
from pydantic import BaseModel, Field
from dspy.functional import TypedPredictor

class Country(BaseModel):
    name: str
    capital: str
    
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer: Country = dspy.OutputField()
 
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = TypedPredictor(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

In [17]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France?")
pred.answer

Country(name='France', capital='Paris')

**We can use the inspect_history to check the conversations**

**Notice DSPy actually corrected LLM to ensure the answer to follow our schema**

**Now we begin to see the power of DSPy**

In [18]:
lm.inspect_history(n=3)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}. Respond with a single JSON object. JSON Schema: {"properties": {"name": {"title": "Name", "type": "string"}, "capital": {"title": "Capital", "type": "string"}}, "required": ["name", "capital"], "title": "Country", "type": "object"}

---

Question: what is the capital city of France?
Answer: Paris. Respond with a single JSON object. JSON Schema: {"properties": {"name": {"title": "Name", "type": "string"}, "capital": {"title": "Capital", "type": "string"}}, "required": ["name", "capital"], "title": "Country", "type": "object"}







Make a very succinct json object that validates with the following schema

---

Follow the following format.

Json Schema: ${json_schema}
Json Object: ${json_object}

---

Json Schema: {"properties": {"name": {"title": "Name", "type": "string"}, "capital": {"title": "Capital", "type": "string"}}, "required": ["name", "cap

## Now Let's Go Further ...

let's try the chain-of-thought thing - in this case, instead of asking the capital city of the country, we want the capital cities of France's neighbor coutries.

In [27]:
class Country(BaseModel):
    name: str
    capital: str
    population: int
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer: Country = dspy.OutputField()
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = dspy.ChainOfThought(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

In [28]:
qa_bot = BasicQABot()
pred = qa_bot.forward("which of France's neighbor country has the most population?")
pred.answer

"Germany has the most population among France's neighboring countries."

In [30]:
lm.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: which of France's neighbor country has the most population?
Reasoning: Let's think step by step in order to produce the answer. We know that France shares borders with several countries, including Spain, Italy, Germany, Switzerland, Belgium, and Luxembourg. By looking at the population statistics of these countries, we can determine which one has the most population.

Answer: Germany has the most population among France's neighboring countries.





Notice the promopt has one new field "reasoning: Let's think step by step..."

Unfortunately we lost the JSON format since we replaced "TypedPredictor" with "ChainOfThought".

**Turns out there is a "TypedChainOfThought" that does both**

In [43]:
from dspy.functional import TypedChainOfThought
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = TypedChainOfThought(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

In [44]:
qa_bot = BasicQABot()
pred = qa_bot.forward("which of France's neighbor country has the most population?")
pred.answer

Country(name='Germany', capital='Berlin', population=83783942)

In [45]:
lm.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}. Respond with a single JSON object. JSON Schema: {"properties": {"name": {"title": "Name", "type": "string"}, "capital": {"title": "Capital", "type": "string"}, "population": {"title": "Population", "type": "integer"}}, "required": ["name", "capital", "population"], "title": "Country", "type": "object"}

---

Question: which of France's neighbor country has the most population?
Reasoning: Let's think step by step in order to produce the answer. We first need to identify France's neighboring countries, which are Spain, Italy, Germany, Switzerland, Luxembourg, Belgium, and Monaco. Next, we need to determine the population of each of these countries and compare them to find the one with the highest population.

Answer: {"name": "Germany", "capital": "Berlin", "population": 837

## Multi-Hop

Search the examples in DSPy, we can find Multi-Hop examples like https://github.com/stanfordnlp/dspy/blob/fc664d5e339d2c16b3b537bdbb13d9707e6fd9c4/skycamp2023.ipynb#L306. 

In above example, it has 3 steps:
1. given a question, ask LLM to return a search query
2. using a "retriever" from DSPy, run the search
3. given the search results, ask LLM to return answer

Since we want to focus on the "prompt" part, we will do #1 and #3, and just hard code the search results like below

In [62]:

class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate_query = dspy.ChainOfThought("question -> search_query")
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self,question):
        # generate a search query from the question, and use it to retrieve passages
        search_query = self.generate_query(question=question).search_query
        print("search:", search_query)
        # here we are supposely to do the search and pass the search results to context.
        passages = """
            country, population
            Germany, 10000000
            Belgium, 10000000000
            Spain, 100000
        """
        print("search results", passages)
        # generate an answer from the passages and the question
        return self.generate_answer(context=passages, question=question)

In [63]:
qa_bot = BasicQABot()
pred = qa_bot.forward("which of France's neighbor country has the most population?")
pred.answer

search: France neighboring countries population statistics
search results 
            country, population
            Germany, 10000000
            Belgium, 10000000000
            Spain, 100000
        


'Belgium, with a population of 10,000,000, is the neighbor country of France with the most population.'

In above exercise, we are using the magical signature "context, question -> answer". It works, but unfortunately we are not getting JSON output since we are not specifying the answer type as before
```
class BasicQA(dspy.Signature):
    ...
    answer: Country = dspy.OutputField()
```

## So What is Signature Anyway?

So far we have seen the magic string like "question -> answer", "question -> search_query", "context, question -> answer", they are referred as "Signature" in DSPy. 

self.generate_query = dspy.ChainOfThought("question -> search_query")

The "dspy.ChainOfThought" is a Predict, let's take a look at it - https://github.com/stanfordnlp/dspy/blob/fc664d5e339d2c16b3b537bdbb13d9707e6fd9c4/docs/api/modules/Predict.md

```
class Predict(Parameter):
    def __init__(self, signature, **config):
        self.stage = random.randbytes(8).hex()
        self.signature = signature
        self.config = config
        self.reset()

        if isinstance(signature, str):
            inputs, outputs = signature.split("->")
            inputs, outputs = inputs.split(","), outputs.split(",")
            inputs, outputs = [field.strip() for field in inputs], [field.strip() for field in outputs]

            assert all(len(field.split()) == 1 for field in (inputs + outputs))

            inputs_ = ', '.join([f"`{field}`" for field in inputs])
            outputs_ = ', '.join([f"`{field}`" for field in outputs])

            instructions = f"""Given the fields {inputs_}, produce the fields {outputs_}."""

            inputs = {k: InputField() for k in inputs}
            outputs = {k: OutputField() for k in outputs}

            for k, v in inputs.items():
                v.finalize(k, infer_prefix(k))
            
            for k, v in outputs.items():
                v.finalize(k, infer_prefix(k))

            self.signature = dsp.Template(instructions, **inputs, **outputs)
```

**So, the signature specifies the inputs and outputs. When it is string like 'context, question -> answer', DSPy will parse it through the '->' and ',' to decide the fields. Otherwise, it can be defined with code like the BasicQA**

In [64]:
class GenerateSearchQuery(dspy.Signature):
    question = dspy.InputField()
    search_query: str = dspy.OutputField()
    
class GenerateAnswer(dspy.Signature):
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer: Country = dspy.OutputField()

class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate_query = TypedChainOfThought(GenerateSearchQuery)
        self.generate_answer = TypedChainOfThought(GenerateAnswer)

    def forward(self,question):
        # generate a search query from the question, and use it to retrieve passages
        search_query = self.generate_query(question=question).search_query
        print("search:", search_query)
        # here we are supposely to do the search and pass the search results to context.
        passages = """
            country, population
            Germany, 10000000
            Belgium, 10000000000
            Spain, 100000
        """
        print("search results", passages)
        # generate an answer from the passages and the question
        return self.generate_answer(context=passages, question=question)

In [65]:
qa_bot = BasicQABot()
pred = qa_bot.forward("which of France's neighbor country has the most population?")
pred.answer

search: France neighboring countries population statistics
search results 
            country, population
            Germany, 10000000
            Belgium, 10000000000
            Spain, 100000
        


Country(name='Belgium', capital='Brussels', population=10000000000)

**Now we have JSON results**

## Thoughts

1. **Loving DSPy for Prompt Help**: It's awesome that DSPy is tackling the dark arts of LLM prompting so I don't have to.
2. **Exploring More Down the Road**: There's a bunch more to this whole topic. We only touched the very basic so far.
3. **Another framework?**: Just like with other frameworks, there's a specific way to code to get the most out of it. I get why it's called the "PyTorch of prompting" and all, it has the vibe of the "ORM-for-Prompt"

I stepped upon this at https://github.com/stanfordnlp/dspy/issues/253, and I think it presents the architecture of DPSy very well.
![diagram](./from_issue_253.png)

**Next, we will explore the feedback loops - the optimizor, assertions**